In [1]:
import nltk
import pandas as pd 
import numpy as np
import string
import re

In [2]:
df = pd.read_csv('contoh.csv')

In [3]:
df

,Tweets
0,Baru semalam membahas betapa Search di beberap...


In [4]:
def case_fold(tweet):
    tweet = tweet.lower()
    return tweet

In [12]:
def remove(tweet):
    #remove angka
    tweet = re.sub('[0-9]+', '', tweet)
    
    # remove tab, new line, ans back slice
    tweet = tweet.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
 
    # remove non ASCII (emoticon, chinese word, .etc)
    tweet = tweet.encode('ascii', 'replace').decode('ascii')
    
    # remove mention, link, hashtag
    tweet = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", tweet).split())

    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    
    #remove punctuation
    tweet = tweet.translate(str.maketrans("","",string.punctuation))
    
    #remove whitespace leading & trailing
    tweet = tweet.strip()
    
    #remove multiple whitespace into single whitespace
    tweet = re.sub('\s+',' ',tweet)

    return tweet

df["Filtering"] = df["Case_Folding"].apply(remove)

In [18]:
from nltk.corpus import stopwords
stop_words=stopwords.words('indonesian')
stop_words.extend(["yg", "dg", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh',
                       '&amp', 'yah', 'id'])

df['without_stopwords'] = df['Filtering'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [19]:
df.to_csv('contoh.csv')

In [20]:
from nltk.tokenize import word_tokenize

df['Tokenizing'] = df['without_stopwords'].apply(word_tokenize)

In [21]:
df

,Tweets,Case_Folding,Filtering,without_stopwords,Tokenizing
0,Baru semalam membahas betapa Search di beberap...,baru semalam membahas betapa search di beberap...,baru semalam membahas betapa search di beberap...,semalam membahas betapa search aplikasi bagus ...,"[semalam, membahas, betapa, search, aplikasi, ..."


In [24]:
import Sastrawi
import numpy as np

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

df['Stemmed'] = df['without_stopwords'].apply(lambda x : ([stemmer.stem(y) for y in x.split()]))

In [26]:
df.to_csv('contoh.csv')

In [28]:
df

,Tweets,Case_Folding,Filtering,without_stopwords,Tokenizing,Stemmed
0,Baru semalam membahas betapa Search di beberap...,baru semalam membahas betapa search di beberap...,baru semalam membahas betapa search di beberap...,semalam membahas betapa search aplikasi bagus ...,"[semalam, membahas, betapa, search, aplikasi, ...","[malam, bahas, betapa, search, aplikasi, bagus..."


In [29]:
import seaborn as sns
import matplotlib.pyplot as plt

from wordcloud import WordCloud

word_dict = {}
for i in range(0,len(df['without_stopwords'])):
    sentence = df['without_stopwords'][i]
    word_token = word_tokenize(sentence)
    for j in word_token:
        if j not in word_dict:
            word_dict[j] = 1
        else:
            word_dict[j] += 1

In [30]:
negasi = ['bukan','tidak','ga','gk']
lexicon = pd.read_csv('biar tenang/data_coba/full_lexicon.csv')
lexicon = lexicon.drop(lexicon[(lexicon['word'] == 'bukan')
                               |(lexicon['word'] == 'tidak')
                               |(lexicon['word'] == 'ga')|(lexicon['word'] == 'gk') ].index,axis=0)
lexicon = lexicon.reset_index(drop=True)

In [31]:
lexicon_word = lexicon['word'].to_list()
lexicon_num_words = lexicon['number_of_words']

ns_words = []
factory = StemmerFactory()
stemmer = factory.create_stemmer()
for word in word_dict.keys():
    if word not in lexicon_word:
        kata_dasar = stemmer.stem(word)
        if kata_dasar not in lexicon_word:
            ns_words.append(word)
len(ns_words)

7

In [32]:
sencol =[]
senrow =np.array([])
nsen = 0

sentiment_list = []
# function to write the word's sentiment if it is founded
def found_word(ind,words,word,sen,sencol,sentiment,add):
    # if it is already included in the bag of words matrix, then just increase the value
    if word in sencol:
        sen[sencol.index(word)] += 1
    else:
    #if not, than add new word
        sencol.append(word)
        sen.append(1)
        add += 1
    #if there is a negation word before it, the sentiment would be the negation of it's sentiment
    if (words[ind-1] in negasi):
        sentiment += -lexicon['weight'][lexicon_word.index(word)]
    else:
        sentiment += lexicon['weight'][lexicon_word.index(word)]
    
    return sen,sencol,sentiment,add
            
# checking every words, if they are appear in the lexicon, and then calculate their sentiment if they do
for i in range(len(df)):
    nsen = senrow.shape[0]
    words = word_tokenize(df['without_stopwords'][i])
    sentiment = 0 
    add = 0
    prev = [0 for ii in range(len(words))]
    n_words = len(words)
    if len(sencol)>0:
        sen =[0 for j in range(len(sencol))]
    else:
        sen =[]
    
    for word in words:
        ind = words.index(word)
        # check whether they are included in the lexicon
        if word in lexicon_word :
            sen,sencol,sentiment,add= found_word(ind,words,word,sen,sencol,sentiment,add)
        else:
        # if not, then check the root word
            kata_dasar = stemmer.stem(word)
            if kata_dasar in lexicon_word:
                sen,sencol,sentiment,add= found_word(ind,words,kata_dasar,sen,sencol,sentiment,add)
        # if still negative, try to match the combination of words with the adjacent words
            elif(n_words>1):
                if ind-1>-1:
                    back_1    = words[ind-1]+' '+word
                    if (back_1 in lexicon_word):
                        sen,sencol,sentiment,add= found_word(ind,words,back_1,sen,sencol,sentiment,add)
                    elif(ind-2>-1):
                        back_2    = words[ind-2]+' '+back_1
                        if back_2 in lexicon_word:
                            sen,sencol,sentiment,add= found_word(ind,words,back_2,sen,sencol,sentiment,add)
    # if there is new word founded, then expand the matrix
    if add>0:  
        if i>0:
            if (nsen==0):
                senrow = np.zeros([i,add],dtype=int)
            elif(i!=nsen):
                padding_h = np.zeros([nsen,add],dtype=int)
                senrow = np.hstack((senrow,padding_h))
                padding_v = np.zeros([(i-nsen),senrow.shape[1]],dtype=int)
                senrow = np.vstack((senrow,padding_v))
            else:
                padding =np.zeros([nsen,add],dtype=int)
                senrow = np.hstack((senrow,padding))
            senrow = np.vstack((senrow,sen))
        if i==0:
            senrow = np.array(sen).reshape(1,len(sen))
    # if there isn't then just update the old matrix
    elif(nsen>0):
        senrow = np.vstack((senrow,sen))
        
    sentiment_list.append(sentiment)

In [33]:
sencol.append('sentiment')
sentiment_array = np.array(sentiment_list).reshape(senrow.shape[0],1)
sentiment_data = np.hstack((senrow,sentiment_array))
df_sen = pd.DataFrame(sentiment_data,columns = sencol)
df_sen

,malam,membahas,betapa,aplikasi,bagus,menyebut,terdekat,great,tim,sentiment
0,1,1,1,1,1,1,1,1,1,5


In [35]:
cek_df = pd.DataFrame([])
cek_df['text'] = df['without_stopwords'].copy()
cek_df['sentiment']  = df_sen['sentiment'].copy()

cek_df

,text,sentiment
0,semalam membahas betapa search aplikasi bagus ...,5


In [37]:
df['sentiment'] = df_sen['sentiment']
df.loc[df['sentiment'] == 0, 'label'] = 'netral'
df.loc[df['sentiment'] > 0, 'label'] = 'positif'
df.loc[df['sentiment'] < 0, 'label'] = 'negatif'
df['label'].value_counts()

positif    1
Name: label, dtype: int64

In [38]:
df

,Tweets,Case_Folding,Filtering,without_stopwords,Tokenizing,Stemmed,sentiment,label
0,Baru semalam membahas betapa Search di beberap...,baru semalam membahas betapa search di beberap...,baru semalam membahas betapa search di beberap...,semalam membahas betapa search aplikasi bagus ...,"[semalam, membahas, betapa, search, aplikasi, ...","[malam, bahas, betapa, search, aplikasi, bagus...",5,positif
